In [26]:
import geopandas as gpd
import pandas as pd
import numpy as np
import re
from ast import literal_eval
import requests
from bs4 import BeautifulSoup

BASE = ''
COMP_BASE = ''

train = pd.read_csv(COMP_BASE + 'train.csv')
test = pd.read_csv(COMP_BASE + 'test.csv')

# Sample
train = pd.read_csv(COMP_BASE + 'train.csv')
test = pd.read_csv(COMP_BASE + 'test.csv')

reveal = pd.read_csv('revealed_test.csv')
#reveal = reveal.set_index('row_id')
train = pd.concat([train,reveal],axis=0).sort_index()

IDS = train['cfips'].unique()
train = train.sort_values(['cfips','first_day_of_month']).reset_index(drop=True)


In [23]:
# Read file into Geopandas
gdf = gpd.read_file(filename = BASE + "raw_data/US_County_Boundaries/US_CountyBndrys.shp")
gdf = gdf.sort_values('GEOID').reset_index(drop=True)
gdf['GEOID'] = gdf['GEOID'].astype(int)

In [24]:
# Read file into Geopandas
gdf = gpd.read_file(filename = BASE + "raw_data/US_County_Boundaries/US_CountyBndrys.shp")
gdf = gdf.sort_values('GEOID').reset_index(drop=True)
gdf['GEOID'] = gdf['GEOID'].astype(int)

# add neighbours column
gdf["neighbours"] = None  

for i, (index, row) in enumerate(gdf.iterrows()):  
    # get 'not disjoint' countries
    neighbors = gdf[~gdf.geometry.disjoint(row.geometry)]['GEOID'].tolist()

    # remove own name of the country from the list
    neighbors = [int(id) for id in neighbors if row.GEOID != id]

    # Insert neighbours to df
    gdf.at[index, "neighbours"] = neighbors

    # Log progress
    if i>0 and i%500==0:
        print("{}/{}".format(i, len(gdf)))

# Selecting relevent cfips
gdf = gdf[['GEOID', 'neighbours','INTPTLAT', 'INTPTLON', 'ALAND']].rename(columns={'GEOID':'cfips', 'INTPTLAT':'lat', 'INTPTLON':'long'})
gdf = gdf[gdf['cfips'].isin(IDS)].reset_index(drop=True)

# Save to disk
gdf.to_csv('county_neighbours.csv', index=False)

print(gdf.shape)
gdf.head()

500/3233
1000/3233


KeyboardInterrupt: 

## Coastline Data

Next, we can look at the US census data to find counties on a coastline. There was no metadata to go with this file, but I confirmed the correctness of this information using folium.

Source: [Data](https://www2.census.gov/library/stories/2018/08/)

In [7]:
# Loading coastline information
coast_df = pd.read_csv(BASE + 'raw_data/coastline-counties-list.csv')
coast_df = coast_df.rename(columns={'STATE/\nCOUNTY\nFIPS': 'cfips', 'COASTLINE REGION':'coast_region'})
coast_df = coast_df[coast_df['cfips'].isin(IDS)][['cfips']]
coast_df['is_coastal'] = True

# Merging Result
new_rows = [i for i in IDS if i not in coast_df['cfips'].values]
new_df = pd.DataFrame(zip(new_rows, len(new_rows)*[False]), columns=coast_df.columns)
coast_df = coast_df.append(new_df).sort_values('cfips').reset_index(drop=True)

#Save to Disk
coast_df.to_csv('county_coastline.csv', index=False)
print(coast_df.shape)
coast_df.head()

(3135, 2)


C:\Users\ui572274\AppData\Local\Temp\1\ipykernel_18488\3505090038.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coast_df = coast_df.append(new_df).sort_values('cfips').reset_index(drop=True)


,cfips,is_coastal
0,1001,False
1,1003,True
2,1005,False
3,1007,False
4,1009,False


## County Estimated Rent Prices

Next, the 50th Percentile Rent Estimate Data was sourced from the [Office of Policy Development and Research](https://www.huduser.gov/portal/home.html). This source could provide a sense of economic health in each county. The frequency of this dataset is annual and provides a median estimation of rent prices for 1,2 and 3-bedroom rentals each year. 

Some counties in the raw data have multiple estimates, so we take the median estimate for each county each year.

Source: [Data](https://www.huduser.gov/portal/datasets/50per.html#2019)

In [9]:
COUNTY_FILES = [
    'raw_data/US_County_Rent/FY2019_50_County_rev.xlsx',
    'raw_data/US_County_Rent/FY2020_50_County_rev.xlsx', 
    'raw_data/US_County_Rent/FY2021_50_County.xlsx',
    'raw_data/US_County_Rent/FY2022_FMR_50_county_rev.csv',
    'raw_data/US_County_Rent/FY2023_FMR_50_county.csv',
]

COLUMNS = ['cfips', 'rent_50_0', 'rent_50_1', 'rent_50_2', 'rent_50_3', 'year']

all_values = []
for file in COUNTY_FILES:
    # Load file
    if file.endswith('xlsx'):
        df = pd.read_excel(BASE + file)
        df = df.rename(columns={'state':'state_code', 'county':'county_code', 'rent50_0':'rent_50_0', 'rent50_1':'rent_50_1', 'rent50_2':'rent_50_2', 'rent50_3':'rent_50_3'})
    else:
        df = pd.read_csv(BASE + file)

    # Load File
    df['cfips'] = (df['state_code'].astype(str).str.zfill(2) + df['county_code'].astype(str).str.zfill(3)).astype(int)
    df = df.sort_values('cfips').reset_index(drop=True)

    # Filter Rows
    df['year'] = df['year'] = int(re.findall('(?<=Y)[0-9]+', file)[0])
    df = df[COLUMNS]
    all_values.append(df.values)

# Combine and take median for duplicate rows
rent_df = pd.DataFrame(np.vstack(all_values), columns=COLUMNS)
rent_df = rent_df.groupby(['cfips','year']).median().reset_index()
rent_df = rent_df[rent_df['cfips'].isin(IDS)].reset_index(drop=True)

# Write to disk
rent_df.to_csv('county_rent_estimates.csv', index=False)

print(rent_df.shape)
rent_df.head()

(15675, 6)


,cfips,year,rent_50_0,rent_50_1,rent_50_2,rent_50_3
0,1001,2019,620.0,743.0,880.0,1112.0
1,1001,2020,615.0,741.0,876.0,1105.0
2,1001,2021,685.0,820.0,972.0,1229.0
3,1001,2022,687.0,816.0,976.0,1234.0
4,1001,2023,773.0,882.0,1055.0,1340.0


## US Treasury Market Yield

Next, the Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity dataset was sourced from the [Federal Reserve Bank of St Louis](https://research.stlouisfed.org/). The frequency of this dataset is daily. 

Since we are making predictions each month, we select the measurement closest but less than the 15th each month (this is because the competition ends on the 14th), and join this with that in the training data.

Source: [Data](https://fred.stlouisfed.org/series/DGS10)

In [10]:
# Load Data
ust_df = pd.read_csv(BASE + "raw_data/DGS10.csv")
ust_df['DATE'] = pd.to_datetime(ust_df['DATE'])
ust_df['DGS10'] = ust_df['DGS10'].replace(".", np.nan).ffill().astype(float)

# Select last value in each month < day 15 (comepetition ends on the 14th)
ust_df = ust_df[ust_df['DATE'].dt.day < 15].groupby([ust_df['DATE'].dt.year, ust_df['DATE'].dt.month]).tail(1).reset_index(drop=True)
ust_df['DATE'] = ust_df['DATE'].apply(lambda dt: dt.replace(day=1))
ust_df = ust_df.rename(columns={'DATE':'first_day_of_month'})

# Save to disk
ust_df.to_csv('10yr_treasury_rate.csv', index=False)

print(ust_df.shape)
ust_df.head()

(61, 2)


,first_day_of_month,DGS10
0,2018-01-01,2.55
1,2018-02-01,2.91
2,2018-03-01,2.81
3,2018-04-01,2.82
4,2018-05-01,3.00


## State Monthly Employment Levels

Finally, we look at the Seasonally adjusted employment data by state sourced from the [Bureau of Labor Statistics](https://www.bls.gov/). This dataset provides monthly employment information by state.

Navigate to the Downloadable Series Files (TXT) section for the raw data after clicking the link below. 

Source: [Data](https://www.bls.gov/web/laus.supp.toc.htm)

In [33]:
with open(BASE + 'raw_data/ststdsadata.txt') as f:
    lines = f.readlines()[15:]

# removing bad lines
clean_lines = []
for line in lines:
    bad_line = False
    for val in ['\n', '   Los', '   New']:
        if line.startswith(val):
            bad_line = True
            break
    if bad_line:
        continue
    clean_lines.append(line.rstrip())

# Parsing data from remaining lines
all_values = []
last_date = None

for line in clean_lines:
    if line.startswith(' '):
        last_date = line.strip()
        continue

    line_split = re.split("\.{2,}", line)
    state = line_split[0].strip()
    values = [state] + [last_date] + [l.replace(',','') for l in line_split[1].split()]
    all_values.append(values)

# Converting data to a df
new_columns = ['state', 'first_day_of_month', 'non_inst_pop', 'total_pop', 'pct_non_inst_pop', 'employed', 'pct_employed', 'unemployed', 'pct_unemployed'] 
emp_df = pd.DataFrame(data=all_values, columns=new_columns)

# Datatypes and save
emp_df['first_day_of_month'] = pd.to_datetime(emp_df['first_day_of_month'], format='%B %Y')
emp_df['first_day_of_month'] = emp_df['first_day_of_month']
for col in emp_df.columns[2:]:
    emp_df[col] = pd.to_numeric(emp_df[col])

# Save to disk
emp_df.to_csv('state_employment.csv', index=False)

print(emp_df.shape)
emp_df.head()

(28764, 9)


,state,first_day_of_month,non_inst_pop,total_pop,pct_non_inst_pop,employed,pct_employed,unemployed,pct_unemployed
0,Alabama,1976-01-01,2605000,1484555,57.0,1386023,53.2,98532,6.6
1,Alaska,1976-01-01,232000,160183,69.0,148820,64.1,11363,7.1
2,Arizona,1976-01-01,1621000,964120,59.5,865871,53.4,98249,10.2
3,Arkansas,1976-01-01,1536000,889044,57.9,824395,53.7,64649,7.3
4,California,1976-01-01,15621000,9774280,62.6,8875685,56.8,898595,9.2


In [34]:
emp_df

,state,first_day_of_month,non_inst_pop,total_pop,pct_non_inst_pop,employed,pct_employed,unemployed,pct_unemployed
0,Alabama,1976-01-01,2605000,1484555,57.0,1386023,53.2,98532,6.6
1,Alaska,1976-01-01,232000,160183,69.0,148820,64.1,11363,7.1
2,Arizona,1976-01-01,1621000,964120,59.5,865871,53.4,98249,10.2
3,Arkansas,1976-01-01,1536000,889044,57.9,824395,53.7,64649,7.3
4,California,1976-01-01,15621000,9774280,62.6,8875685,56.8,898595,9.2
...,...,...,...,...,...,...,...,...,...
28759,Virginia,2022-12-01,6844170,4357319,63.7,4228407,61.8,128912,3.0
28760,Washington,2022-12-01,6219277,4002539,64.4,3835035,61.7,167504,4.2
28761,West Virginia,2022-12-01,1444732,791977,54.8,759807,52.6,32170,4.1
28762,Wisconsin,2022-12-01,4752266,3075974,64.7,2976802,62.6,99172,3.2


## Tax Rate Data

The final data source provides information on the Top Marginal Corporate Income Tax Rate by state and year. This data was sourced from [Richstatespoorstates](https://www.richstatespoorstates.org/), and could be used as another economic indicator.

Source: [Data](https://www.richstatespoorstates.org/variables/corporate_income_tax_rate/)

## Merging w/ Competition Data

The following cell shows how each cleaned csv file can be merged with the competition data.

In [35]:
# Loading Competition data
#train = pd.read_csv(COMP_BASE + 'train.csv')
test = pd.read_csv(COMP_BASE + 'test.csv')
train['istest'] = 0
test['istest'] = 1

# Merging train and test data
all_data = pd.concat((train, test)).sort_values(['cfips','row_id']).reset_index(drop=True)
all_data['first_day_of_month'] = pd.to_datetime(all_data["first_day_of_month"])

print(all_data.shape)
all_data.head()

(153615, 8)


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249.0,0
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0


In [36]:
# County neighbours data
neigbbour_df = pd.read_csv('county_neighbours.csv')
neigbbour_df['neighbours'] = neigbbour_df['neighbours'].apply(literal_eval)
all_data = pd.merge(all_data, neigbbour_df, on='cfips')

# Coastline data
coast_df = pd.read_csv("county_coastline.csv")
coast_df = coast_df[['cfips', 'is_coastal']]
all_data = pd.merge(all_data, coast_df, on="cfips", how="left")

# Rent estimate data
rent_df = pd.read_csv('county_rent_estimates.csv')
all_data = pd.merge(all_data, rent_df, 
                    left_on=['cfips', all_data['first_day_of_month'].dt.year], 
                    right_on=['cfips', 'year']).drop(columns='year')

# Treasury 10 yr yields
trsy_df = pd.read_csv("10yr_treasury_rate.csv")
trsy_df['first_day_of_month'] = pd.to_datetime(trsy_df['first_day_of_month'])
all_data = pd.merge(all_data, trsy_df, on='first_day_of_month', how='left')

# State employment data
emp_df = pd.read_csv("state_employment.csv")
emp_df['first_day_of_month'] = pd.to_datetime(emp_df['first_day_of_month'])
all_data = pd.merge(all_data, emp_df, on=['state', 'first_day_of_month'], how='left')

# State tax rate data
tax_df = pd.read_csv("state_corporate_tax_rates.csv")
all_data = pd.merge(all_data, tax_df,
                    left_on=['state', all_data['first_day_of_month'].dt.year],
                    right_on=['state', 'year'],
                    how='left')

print(all_data.shape)
all_data.head()

(153615, 27)


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,neighbours,lat,...,non_inst_pop,total_pop,pct_non_inst_pop,employed,pct_employed,unemployed,pct_unemployed,year,tax_rate,tax_rate_diff
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249.0,0,"[1021, 1047, 1051, 1085, 1101]",32.532237,...,3938622.0,2271806.0,57.7,2206223.0,56.0,65583.0,2.9,2019,6.03,0.89
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0,"[1021, 1047, 1051, 1085, 1101]",32.532237,...,3941513.0,2273771.0,57.7,2207294.0,56.0,66477.0,2.9,2019,6.03,0.89
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0,"[1021, 1047, 1051, 1085, 1101]",32.532237,...,3944209.0,2275680.0,57.7,2207568.0,56.0,68112.0,3.0,2019,6.03,0.89
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0,"[1021, 1047, 1051, 1085, 1101]",32.532237,...,3946542.0,2276674.0,57.7,2206498.0,55.9,70176.0,3.1,2019,6.03,0.89
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0,"[1021, 1047, 1051, 1085, 1101]",32.532237,...,3948677.0,2275425.0,57.6,2202912.0,55.8,72513.0,3.2,2019,6.03,0.89


In [37]:
all_data.to_csv('new_train.csv')

In [38]:
all_data[all_data.first_day_of_month == "2022-12-01"].iloc[0]

row_id                                  1001_2022-12-01
cfips                                              1001
county                                   Autauga County
state                                           Alabama
first_day_of_month                  2022-12-01 00:00:00
microbusiness_density                          3.470915
active                                           1475.0
istest                                                0
neighbours               [1021, 1047, 1051, 1085, 1101]
lat                                           32.532237
long                                         -86.646439
is_coastal                                        False
rent_50_0                                         687.0
rent_50_1                                         816.0
rent_50_2                                         976.0
rent_50_3                                        1234.0
DGS10                                              3.49
non_inst_pop                                  40